In [1]:
import pandas as pd
import json
import numpy as np
import codecs

In [2]:
# Handle JSON conversions
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        if isinstance(obj, float) and np.isnan(obj):
          return None
        return super(NpEncoder, self).default(obj)

In [ ]:
!pip install ipdb

import ipdb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.7 MB/s eta 0:00:00


In [3]:
# Load the data into a pandas dataframe:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# load the SG Coarse ONET Dataset
file = "/content/drive/MyDrive/MTECH IS Project/data/jobs_merge_with_soc2018_and_sg_coarse_df.csv"
course_df = pd.read_csv(file, sep=',')

# Remove the Unamed:  0 Column
course_df = course_df.drop(columns=course_df.columns[0])
course_df = course_df[['O*NET-SOC Code', 'Title', 'Job Zone', 'university', 'school', 'degree']]

course_df[course_df.notnull().all(1)].head(120)

,O*NET-SOC Code,Title,Job Zone,university,school,degree
9,11-3012.00,Administrative Services Managers,3.0,Singapore Institute of Technology,DigiPen Institute of Technology,Bachelor of Fine Arts in Digital Arts & Animation
10,11-3012.00,Administrative Services Managers,3.0,Singapore Institute of Technology,DigiPen Institute of Technology,Bachelor of Science in Computer Science & Game...
11,11-3012.00,Administrative Services Managers,3.0,Singapore Institute of Technology,DigiPen Institute of Technology,Bachelor of Science in Computer Science in Rea...
30,11-9013.00,"Farmers, Ranchers, and Other Agricultural Mana...",4.0,Singapore Institute of Technology,DigiPen Institute of Technology,Bachelor of Fine Arts in Digital Art and Anima...
31,11-9013.00,"Farmers, Ranchers, and Other Agricultural Mana...",4.0,Singapore Institute of Technology,DigiPen Institute of Technology,Bachelor of Science in Computer Science and Ga...
...,...,...,...,...,...,...
400,25-1041.00,"Agricultural Sciences Teachers, Postsecondary",5.0,National University of Singapore,Faculty of Engineering,Bachelor of Engineering (Industrial and System...
401,25-1041.00,"Agricultural Sciences Teachers, Postsecondary",5.0,Nanyang Technological University,College of Engineering,Materials Engineering
402,25-1041.00,"Agricultural Sciences Teachers, Postsecondary",5.0,Nanyang Technological University,College of Engineering,Mechanical Engineering
403,25-1041.00,"Agricultural Sciences Teachers, Postsecondary",5.0,Nanyang Technological University,College of Engineering,Mechanical Engineering and Economics **


In [6]:
# Load educational programs data
programs_file = "/content/drive/MyDrive/MTECH IS Project/data/course_graduate_employment/programs_raw.json"
program_infos_file = "/content/drive/MyDrive/MTECH IS Project/data/course_graduate_employment/program_infos.json"


# Opening JSON file
_programs_file = open(programs_file, mode='r', encoding='utf-8-sig')
_program_infos_file = open(program_infos_file, mode='r', encoding='utf-8-sig')
  
# returns JSON object as 
# a dictionary
programs = json.load(_programs_file)
program_infos = json.load(_program_infos_file)
  
# Key the arrays by ID
programs_by_id = { d['id']: d for d in programs }
programs_by_id_json = json.dumps(programs_by_id, cls=NpEncoder)

with open("programs.json", "w") as outfile:
    outfile.write(programs_by_id_json)

programs_infos_by_id = {}
for d in program_infos:
  id = f"{d['program_id']}_{d['year']}"
  d["id"] = id
  programs_infos_by_id[id] = d

programs_infos_by_id_json = json.dumps(programs_infos_by_id, cls=NpEncoder)

with open("program_trends.json", "w") as outfile:
    outfile.write(programs_infos_by_id_json)



In [ ]:
# Load Salary data
file_s = "/content/drive/MyDrive/MTECH IS Project/data/onet_salary.csv"
salary_df = pd.read_csv(file_s, sep=',')

In [ ]:
# Convert IDs to string
salary_df['isco_code'] = salary_df['isco_code'].apply(str)
salary_df['ssoc_code'] = salary_df['ssoc_code'].apply(str)

salary_df.head()

,Unnamed: 0,onetsoc_code,title,description,isco_code,ssoc_code,ssoc_job_title,top_riasec,riasec,riasec_rank,min_salary,max_salary
0,0,11-1011.00,Chief Executives,Determine and formulate policies and provide o...,1112,11121,Ambassador (government),E,"E,C,S","1,2,3",1800,5500
1,1,11-1011.00,Chief Executives,Determine and formulate policies and provide o...,1112,11121,Auditor general,E,"E,C,S","1,2,3",1800,7000
2,2,11-1011.00,Chief Executives,Determine and formulate policies and provide o...,1112,11121,Civil service commissioner,E,"E,C,S","1,2,3",0,0
3,3,11-1011.00,Chief Executives,Determine and formulate policies and provide o...,1112,11121,Commissioner of civil defence force,E,"E,C,S","1,2,3",0,0
4,4,11-1011.00,Chief Executives,Determine and formulate policies and provide o...,1112,11121,Commissioner of inland revenue,E,"E,C,S","1,2,3",0,0


In [ ]:
# Select columns
select_cols = ['onetsoc_code', 'isco_code', 'ssoc_code', 'ssoc_job_title', 'min_salary', 'max_salary']

# Aggregate SSOC codes
def apply_extraction(row):
  column_names = list(row.columns)
  count = len(row)
  result = [None] * count
  for index in range(count):
    current_row = row.iloc[index]
    result[index] = {name: getattr(current_row, name) for name in column_names}
  return result

salary_group_df = salary_df.groupby('ssoc_code')[select_cols].apply(apply_extraction).reset_index(name='jobs')

salary_group_df.head()

,ssoc_code,jobs
0,11110,"[{'onetsoc_code': '11-1031.00', 'isco_code': '..."
1,11121,"[{'onetsoc_code': '11-1011.00', 'isco_code': '..."
2,11122,"[{'onetsoc_code': '11-1011.00', 'isco_code': '..."
3,11140,"[{'onetsoc_code': '11-1021.00', 'isco_code': '..."
4,11150,"[{'onetsoc_code': '11-1021.00', 'isco_code': '..."


In [ ]:
ssoc_columns = ['isco_code', 'ssoc_code', 'ssoc_job_title', 'min_salary', 'max_salary']
ssoc_jobs = {}

for index, row in salary_group_df.iterrows():
  count = len(row["jobs"])

  for i in range(count):
    job = row.jobs[i]
    id = f"{job['ssoc_code']}_{i}"
    ssoc_jobs[id] = {name: job[name] for name in ssoc_columns}
    ssoc_jobs[id]['id'] = id

    # Reassign property as occupation ID
    ssoc_jobs[id]["occupation_id"] = job['onetsoc_code']
    # ipdb.set_trace()


ssoc_json = json.dumps(ssoc_jobs, cls=NpEncoder)

with open("ssoc_jobs.json", "w") as outfile:
    outfile.write(ssoc_json)

In [ ]:
# Aggregate by O*NET SOC codes
select_cols = ['min_salary', 'max_salary']

salary_c_df = salary_df.copy()
# Remove unrealistic small values
salary_c_df[select_cols] = salary_c_df[select_cols].mask(salary_c_df[select_cols] < 100)

onet_group_df = salary_c_df.groupby('onetsoc_code').agg({'min_salary': ['min'], 'max_salary': ['max']})

onet_group_df.head(20)

,min_salary,max_salary
,min,max
onetsoc_code,,
11-1011.00,1000.0,20000.0
11-1011.03,1000.0,20000.0
11-1021.00,480.0,20000.0
11-1031.00,1600.0,8500.0
11-2011.00,1800.0,15000.0
11-2021.00,700.0,19000.0
11-2022.00,700.0,19000.0
11-3021.00,800.0,20000.0


In [ ]:
onet_group_df.iloc(0)[1]['min_salary']['min']

onet_group_df.iloc(0)[1]['min_salary']['min']

1000.0

In [ ]:
onet_salaries = {}

def convert_value(value):
  return None if np.isnan(value) else value

for index, row in onet_group_df.iterrows():
    onet_salaries[index] = {
        "id": index,
        'min_salary': convert_value(row['min_salary']['min']),
        'max_salary': convert_value(row['max_salary']['max'])
    }

onet_salaries_json = json.dumps(onet_salaries, cls=NpEncoder)

with open("onet_salaries.json", "w") as outfile:
    outfile.write(onet_salaries_json)